#### importing data set

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

#### load dataset

In [2]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)
test_dataset = dsets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor())


#### make dataset iterable

In [3]:
batch_size = 100
n_iters=3000
num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs= int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels=16,kernel_size=5, stride=1,padding = 2)
        self.relu1 = nn.ReLU()
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels = 16, out_channels=32,kernel_size=5, stride=1,padding = 2)
        self.relu2 = nn.ReLU()
        
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32*7*7,10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        
        out = out.view(out.size(0),-1)
        
        out = self.fc1(out)
        
        return out
    

        
        

In [5]:
model = CNNModel()

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [10]:
iter = 0 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
                          
        iter += 1
        if iter % 250 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images)
                
                outputs=model(images)
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            accuracy = 100 * correct/total
            
            print('iterations:{}. Accuracy: {}'.format(iter,accuracy))

iterations:250. Accuracy: 97
iterations:500. Accuracy: 97
iterations:750. Accuracy: 98
iterations:1000. Accuracy: 98
iterations:1250. Accuracy: 98
iterations:1500. Accuracy: 98
iterations:1750. Accuracy: 98
iterations:2000. Accuracy: 98
iterations:2250. Accuracy: 98
iterations:2500. Accuracy: 98
iterations:2750. Accuracy: 98
iterations:3000. Accuracy: 98


In [11]:
model = CNNModel()
model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [12]:
iter = 0 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
                          
        iter += 1
        if iter % 250 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.cuda())
                
                outputs=model(images)
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted.cpu() == labels.cpu()).sum()
            accuracy = 100 * correct/total
            
            print('iterations:{}. Accuracy: {}'.format(iter,accuracy))

iterations:250. Accuracy: 96
iterations:500. Accuracy: 97
iterations:750. Accuracy: 97
iterations:1000. Accuracy: 98
iterations:1250. Accuracy: 98
iterations:1500. Accuracy: 98
iterations:1750. Accuracy: 98
iterations:2000. Accuracy: 98
iterations:2250. Accuracy: 98
iterations:2500. Accuracy: 98
iterations:2750. Accuracy: 98
iterations:3000. Accuracy: 98
